In [1]:
import sys
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import euclidean_distances
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import os
sys.path.append(os.path.abspath('../scripts'))
from satisfaction_analysis import (
    load_and_prepare_data,
    treat_missing_and_outliers,
    aggregate_per_customer,
    calculate_engagement_score,
    calculate_experience_score,
    calculate_satisfaction_score,
    build_regression_model,
    run_kmeans,
    aggregate_scores_per_cluster,
    save_to_csv
    )



In [2]:
query = """
SELECT *
FROM xdr_data
"""
df = load_and_prepare_data(query)
df = treat_missing_and_outliers(df)

c:\Users\Tsega KisPay\Documents\GitHub\TellCo-Telecom-Analytics-Dashboard\scripts\load_data.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [3]:
# Aggregate data per customer
user_aggregated_data = aggregate_per_customer(df)
print(user_aggregated_data.head())

   MSISDN/Number  sessions_frequency  total_session_duration  \
0   3.360100e+10                   1                116720.0   
1   3.360100e+10                   1                181230.0   
2   3.360100e+10                   1                134969.0   
3   3.360101e+10                   1                 49878.0   
4   3.360101e+10                   2                 37104.0   

   total_download_data  total_upload_data  avg_tcp_retransmission     avg_rtt  \
0         8.426375e+08         36053108.0            2.080991e+07   46.000000   
1         1.207552e+08         36104459.0            2.080991e+07   30.000000   
2         5.566597e+08         39306820.0            2.080991e+07  109.795706   
3         4.019932e+08         20327526.0            1.066000e+03   69.000000   
4         1.363130e+09         94280527.0            1.507977e+07   57.000000   

                     handset_type  avg_throughput  total_data_volume  
0  Huawei P20 Lite Huawei Nova 3E            37.0       8

In [4]:
# Engagement Clustering
engagement_kmeans = KMeans(n_clusters=3, random_state=42)
user_aggregated_data['engagement_cluster'] = engagement_kmeans.fit_predict(user_aggregated_data[['sessions_frequency', 'total_session_duration', 'total_data_volume']])
print(user_aggregated_data.head())

   MSISDN/Number  sessions_frequency  total_session_duration  \
0   3.360100e+10                   1                116720.0   
1   3.360100e+10                   1                181230.0   
2   3.360100e+10                   1                134969.0   
3   3.360101e+10                   1                 49878.0   
4   3.360101e+10                   2                 37104.0   

   total_download_data  total_upload_data  avg_tcp_retransmission     avg_rtt  \
0         8.426375e+08         36053108.0            2.080991e+07   46.000000   
1         1.207552e+08         36104459.0            2.080991e+07   30.000000   
2         5.566597e+08         39306820.0            2.080991e+07  109.795706   
3         4.019932e+08         20327526.0            1.066000e+03   69.000000   
4         1.363130e+09         94280527.0            1.507977e+07   57.000000   

                     handset_type  avg_throughput  total_data_volume  \
0  Huawei P20 Lite Huawei Nova 3E            37.0       

In [5]:
# Experience Clustering
experience_kmeans = KMeans(n_clusters=3, random_state=42)
user_aggregated_data['experience_cluster'] = experience_kmeans.fit_predict(user_aggregated_data[['avg_tcp_retransmission', 'avg_rtt', 'avg_throughput']])
print(user_aggregated_data.head())

   MSISDN/Number  sessions_frequency  total_session_duration  \
0   3.360100e+10                   1                116720.0   
1   3.360100e+10                   1                181230.0   
2   3.360100e+10                   1                134969.0   
3   3.360101e+10                   1                 49878.0   
4   3.360101e+10                   2                 37104.0   

   total_download_data  total_upload_data  avg_tcp_retransmission     avg_rtt  \
0         8.426375e+08         36053108.0            2.080991e+07   46.000000   
1         1.207552e+08         36104459.0            2.080991e+07   30.000000   
2         5.566597e+08         39306820.0            2.080991e+07  109.795706   
3         4.019932e+08         20327526.0            1.066000e+03   69.000000   
4         1.363130e+09         94280527.0            1.507977e+07   57.000000   

                     handset_type  avg_throughput  total_data_volume  \
0  Huawei P20 Lite Huawei Nova 3E            37.0       

In [6]:
 #Calculate engagement and experience scores
user_aggregated_data = calculate_engagement_score(user_aggregated_data, engagement_kmeans)
user_aggregated_data = calculate_experience_score(user_aggregated_data, experience_kmeans)
print(user_aggregated_data.head())

   MSISDN/Number  sessions_frequency  total_session_duration  \
0   3.360100e+10                   1                116720.0   
1   3.360100e+10                   1                181230.0   
2   3.360100e+10                   1                134969.0   
3   3.360101e+10                   1                 49878.0   
4   3.360101e+10                   2                 37104.0   

   total_download_data  total_upload_data  avg_tcp_retransmission     avg_rtt  \
0         8.426375e+08         36053108.0            2.080991e+07   46.000000   
1         1.207552e+08         36104459.0            2.080991e+07   30.000000   
2         5.566597e+08         39306820.0            2.080991e+07  109.795706   
3         4.019932e+08         20327526.0            1.066000e+03   69.000000   
4         1.363130e+09         94280527.0            1.507977e+07   57.000000   

                     handset_type  avg_throughput  total_data_volume  \
0  Huawei P20 Lite Huawei Nova 3E            37.0       

In [7]:
#Calculate satisfaction score
user_aggregated_data = calculate_satisfaction_score(user_aggregated_data)
top_10_satisfied_customers = user_aggregated_data.nlargest(10, 'satisfaction_score')
print("Top 10 Satisfied Customers:\n", top_10_satisfied_customers)

Top 10 Satisfied Customers:
         MSISDN/Number  sessions_frequency  total_session_duration  \
106853   4.188282e+10                1066            6.874284e+07   
6437     3.361489e+10                  17            2.257121e+06   
92923    3.376054e+10                  15            2.228694e+06   
13180    3.362578e+10                  17            2.088903e+06   
13526    3.362632e+10                  18            2.579965e+06   
76363    3.367588e+10                  15            3.034512e+06   
37052    3.365973e+10                  16            3.643864e+06   
63028    3.366646e+10                  11            2.100622e+06   
92577    3.376041e+10                  12            1.799370e+06   
57241    3.366471e+10                  11            2.927785e+06   

        total_download_data  total_upload_data  avg_tcp_retransmission  \
106853         4.869236e+11       4.436642e+10            9.781259e+06   
6437           8.156743e+09       6.894830e+08            7.540

In [8]:
# Build regression model
model = build_regression_model(user_aggregated_data)
print("Regression Model Coefficients:\n", model.coef_)


Regression Model Coefficients:
 [0.5 0.5]


In [9]:
# Run k-means clustering on engagement and experience scores
user_aggregated_data, kmeans = run_kmeans(user_aggregated_data)
print(user_aggregated_data.head())

   MSISDN/Number  sessions_frequency  total_session_duration  \
0   3.360100e+10                   1                116720.0   
1   3.360100e+10                   1                181230.0   
2   3.360100e+10                   1                134969.0   
3   3.360101e+10                   1                 49878.0   
4   3.360101e+10                   2                 37104.0   

   total_download_data  total_upload_data  avg_tcp_retransmission     avg_rtt  \
0         8.426375e+08         36053108.0            2.080991e+07   46.000000   
1         1.207552e+08         36104459.0            2.080991e+07   30.000000   
2         5.566597e+08         39306820.0            2.080991e+07  109.795706   
3         4.019932e+08         20327526.0            1.066000e+03   69.000000   
4         1.363130e+09         94280527.0            1.507977e+07   57.000000   

                     handset_type  avg_throughput  total_data_volume  \
0  Huawei P20 Lite Huawei Nova 3E            37.0       

In [10]:
# Aggregate scores per cluster
cluster_stats = aggregate_scores_per_cluster(user_aggregated_data)
print("Cluster Statistics:\n", cluster_stats)


Cluster Statistics:
    cluster  engagement_score  experience_score  satisfaction_score
0        0      3.443900e+08      6.806882e+07        2.062294e+08
1        1      5.307569e+11      7.283117e+07        2.654149e+11


In [11]:
# Export to csv
save_to_csv(user_aggregated_data)
print("Data saved to CSV file in 'data' directory.")

Data saved to CSV file in 'data' directory.
